In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import HeUniform
import zipfile
import scipy.io as scio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as ps
import numpy as np

In [ ]:
# unzip data
with zipfile.ZipFile('diabetes.zip', 'r') as zip_ref:
    zip_ref.extractall('./')
with zipfile.ZipFile('sinc.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
dat = open('diabetes2.dt', 'r').read().splitlines()[7:]
for i, r in enumerate(dat):
    dat[i] = [float(v) for v in r.split(' ')]
dat = np.array(dat)

print(sum(dat[:,-1])/len(dat))
print(len(dat))
print(len(dat)*len(dat[0]))

In [ ]:
# split data
train, test = train_test_split(dat, test_size = 0.1, random_state = 44)
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]


print(sum(train_y)/len(train))
print(len(train))
print(sum(test_y)/len(test_y))
print(len(test_y))

In [ ]:
# normalize data
scaler = StandardScaler()
scaler = scaler.fit(train_x)

train_x_norm = scaler.transform(train_x)
test_x_norm = scaler.transform(test_x)

In [ ]:
# prepare architecture
n = len(train_x_norm[0])
m = len(train_x_norm)

he = HeUniform(seed=43)
model = Sequential([
    Input(shape = (n,)),
    
    Dense(n*2, kernel_initializer=he),
    LeakyReLU(),
    
    Dense(n*3, kernel_initializer=he),
    LeakyReLU(),
    
    Dense(1, activation='sigmoid', kernel_initializer=HeUniform)
])

In [ ]:
# ELM modification
for layer in model.layers[:-1]:
    layer.trainable = False

In [ ]:
# Stocastic Gradient descent for Online - ELM
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model.fit(train_x, train_y, shuffle=True, validation_split = 0.1, batch_size = 1, epochs = 2)

In [ ]:
model.evaluate(train_x_norm, train_y, batch_size = 1)
model.evaluate(test_x_norm, test_y, batch_size = 1)

In [ ]:
# compare to single layer perceptron
model = Sequential([
    Input(shape=(n,)),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_x_norm, train_y, validation_split=0.1, batch_size = 1, epochs = 1)
model.evaluate(train_x_norm, train_y)
model.evaluate(test_x_norm, test_y)